# Spatial plots of CWV and CAPE

From ERA-5

In [1]:
import geopy.distance
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import gridspec
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import numpy as np
import xarray as xr
import json
import cartopy
import pandas as pd
from datetime import datetime, timedelta
import cftime
#from adjustText import adjust_text
import glob
import os

### Set region to plot and start and end time

In [2]:
lonMin, lonMax = -62.1, -9.9
latMin, latMax = -2.1, 22.1
#latMin, latMax = -10.1,30.1

start_mon = 'Aug.'
start_day = '10'
start_time = '08-'+start_day+'T00:00:00'
end_mon = 'Sep.'
end_day = '30'
end_time = '09-'+end_day+'T00:00:00'

### Plot CWV

In [ ]:
# For each year, spatial plot of CWV

fig = plt.figure(figsize = (20, 20))
gs = gridspec.GridSpec(6, 5, width_ratios=[1, 1, 1, 1, 1])

# Filename base for ERA-5 CWV data
filebase_CWV = "/huracan/tank4/cornell/ORCESTRA/era5/total_column_water_vapour/"

years = np.arange(1996,2025)  # 1998-2024
iyear = 0
for yy in years:
    # print year
    print("Processing year:", yy)
    
    # Load ERA-5 CWV data
    file_paths_CWV_08 = glob.glob(filebase_CWV + str(yy) + '08/' + "*.nc")
    file_paths_CWV_09 = glob.glob(filebase_CWV + str(yy) + '09/' + "*.nc")
    file_paths_CWV = file_paths_CWV_08 + file_paths_CWV_09
    era5_cwv = xr.open_mfdataset(file_paths_CWV,combine='by_coords')
    
    #Extract for time period of campaign and region
    cwv_yy = era5_cwv.sel(latitude=slice(latMax,latMin),longitude=slice(lonMin,lonMax),valid_time=slice(str(yy)+'-'+start_time,str(yy)+'-'+end_time))

    #Take mean over campaign
    cwv_yy_mean = cwv_yy.mean(dim='valid_time')
    
    # Create plot
    ax1 = fig.add_subplot(gs[iyear],projection=ccrs.PlateCarree())
    ax1.coastlines(resolution = '50m',alpha=0.5)
    #ax1.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha = 0.25)
    gl = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1,alpha = 0.25)
    gl.top_labels = False
    gl.left_labels = False
    if iyear==25 or iyear==26 or iyear==27 or iyear==28:
        print(iyear)    
        gl.xlocator = mticker.FixedLocator([-60, -50, -40, -30, -20, -10])
        gl.xformatter = LONGITUDE_FORMATTER
        gl.xlabel_style = {'size': 14}
    else: 
        gl.bottom_labels = False
    if iyear==4 or iyear==9 or iyear==14 or iyear==19 or iyear==24 or iyear==28:
        print(iyear)
        gl.ylocator = mticker.FixedLocator([0, 5, 10, 15, 20])
        gl.yformatter = LATITUDE_FORMATTER
        gl.ylabel_style = {'size': 14}
    else:
        gl.right_labels = False
    
    # Plot this years CWV as filled contours
    plt.contourf(cwv_yy_mean.longitude.values, 
                 cwv_yy_mean.latitude.values,
                 cwv_yy_mean.tcwv.values,
                 cmap = 'viridis',vmin=25,vmax=60,extend='both')

    if iyear==0 or iyear==5 or iyear==10 or iyear==15 or iyear==20 or iyear==25:
        cbar=plt.colorbar(location='left',orientation='vertical',pad=0.02)
        cbar.set_label('CWV (kg m$^{-2}$)',fontsize=14)

    # Set the plot limits and title
    ax1.set_title(str(yy),size=14)
    ax1.set_xlim(lonMin, lonMax)
    ax1.set_ylim(latMin, latMax)
    
    era5_cwv.close()
    cwv_yy.close()
    iyear=iyear+1
    
sns.set_context('paper') 
plt.tight_layout()  # Adjust subplots to fit into figure area.

Processing year: 1996
Processing year: 1997


In [ ]:
era5_cwv

<xarray.Dataset> Size: 1GB
Dimensions:     (valid_time: 1008, latitude: 241, longitude: 1440)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 8kB 2024-08-15 ... 2024-09-25T23:...
  * latitude    (latitude) float64 2kB 30.0 29.75 29.5 ... -29.5 -29.75 -30.0
  * longitude   (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
    expver      (valid_time) <U4 16kB dask.array<chunksize=(24,), meta=np.ndarray>
Data variables:
    tcwv        (valid_time, latitude, longitude) float32 1GB dask.array<chunksize=(12, 121, 720), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-21T20:23 GRIB to CDM+CF via cfgrib-0.9.1...